# BERT From Scratch

### 1- Importing Dependencies

In [13]:
import math # Mathematics
import re  # Regular expersions
from random import *  # Random values
import numpy as np  # Matrices
import torch  # Transformer # TF # JAX >>> google package Enable us to build BERT too
import torch.nn as nn
import torch.optim as optim  # Optimizers
from transformers import BertTokenizer, BertModel, AdamW

### 2- Creating - **Loading** Our Data

In [14]:
text = (
        'Hello, how are you? I am Ali.\n'
        'Hello, Ali My name is wael. Nice to meet you.\n'
        'Nice meet you too. How are you today?\n'
        'Great. My baseball team won the competition.\n'
        'Oh Congratulations, wael\n'
        'Thank you Ali'
    )

## 3- Data Preprocessing

1. Clean Data from .,?!
2. Encode our data (ids).



In [15]:
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'

word_list = list(set(" ".join(sentences).split())) # we make a join for all sentence to be as one sentense then split by " " and make it in set to remove redundant words the in a list
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


for i, w in enumerate(word_list):
    word_dict[w] = i + 4          # For Encoding start from 4

number_dict = {i: w for i, w in enumerate(word_dict)} # Create a reverse dictionary for decoding
vocab_size = len(word_dict)

token_list = list()
# Tokenize each sentence into a list of token IDs
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    # print("arr = ", arr)
    token_list.append(arr)

### 4. Prepare our Data for BERT.
1. Configure BERT Parameters.
2. Make Batches.

In [16]:
batch_size = 6
maxlen = 30 # maximum of length
max_pred = 5  # max tokens of prediction
n_layers = 6 # number of Encoder of Encoder Layer
n_heads = 12 # number of heads in Multi-Head Attention
d_model = 768 # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V        64*12 head = 768
n_segments = 2 # n.o.sentences

In [17]:

def make_batch():
    batch = []
    positive = negative = 0
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))  # random index , random index
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]  # random sentence , random sentence

        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']] # concatenate random sentences with 'CLS' and 'SEP'

        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)  # [ 0,0,0,0,0,0,1,1,1,1,1,1,1,1,1]

        #MASK LM
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # number of masks is up to 15 % of tokens in one sentence

        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]  # list of indices of input without cls and sep tokens

        shuffle(cand_maked_pos)    # shuffle all indices
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)     # list of random indices
            masked_tokens.append(input_ids[pos]) # list of random tokens
            if random() < 0.8:  # 80%
                input_ids[pos] = word_dict['[MASK]'] # make mask
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word_dict[number_dict[index]] # replace by random word

        # Zero Paddings
        n_pad = maxlen - len(input_ids) # number of pad tokens we need
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

    #     # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred: # target 1 , max = 5   [12,0,0,0,0 ]
            n_pad = max_pred - n_pred # number of pad tokens in masked list
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)  # to make masked list is static in shape
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
    return batch



### 5. Core Functions
1. Get Attention Pad Mask.
2. GELU.

In [18]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token / searching for pad tokens
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

In [19]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [20]:
batch = make_batch()

In [21]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch)) # convert to torch tensor

In [22]:
print(get_attn_pad_mask(input_ids, input_ids)[0][0], input_ids[0])

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True]) tensor([ 1, 21,  3,  5, 22,  3, 10,  5,  6,  2, 11, 16,  3, 25, 12, 15, 14,  2,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


### 6. Building Blocks
1. Embedding.
2. Multi Head Attention.
3. Feed Forward
4. Encoder Block

In [23]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding # size of dictionary, embed vector size
        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

In [24]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

In [25]:
emb = Embedding()
embeds = emb(input_ids, segment_ids)

attenM = get_attn_pad_mask(input_ids, input_ids)

SDPA= ScaledDotProductAttention()(embeds, embeds, embeds, attenM)
C, A = SDPA

print('Masks',attenM[0][0])
print()
print('nnAttention Scores after softmax: ', A[0][0])

Masks tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True])

nnAttention Scores after softmax:  tensor([1.0000e+00, 6.9790e-27, 3.9032e-30, 2.9727e-28, 2.1375e-27, 7.5038e-30,
        3.5430e-27, 2.4213e-28, 7.2006e-29, 1.0190e-26, 7.7632e-43, 1.4781e-41,
        1.4013e-45, 8.7301e-43, 4.2039e-45, 2.9427e-44, 4.5682e-43, 1.5947e-42,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       grad_fn=<SelectBackward0>)


In [26]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]


In [27]:
emb = Embedding()
embeds = emb(input_ids, segment_ids)

attenM = get_attn_pad_mask(input_ids, input_ids)

MHA= MultiHeadAttention()(embeds, embeds, embeds, attenM)

Output, A = MHA

A[0][0]

tensor([[0.0700, 0.0592, 0.0682, 0.0480, 0.0831, 0.0651, 0.0727, 0.0426, 0.0636,
         0.0688, 0.0676, 0.0488, 0.0392, 0.0364, 0.0296, 0.0500, 0.0488, 0.0384,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0800, 0.0513, 0.0467, 0.0434, 0.0409, 0.0546, 0.0620, 0.0465, 0.0598,
         0.0557, 0.0572, 0.0495, 0.0760, 0.0679, 0.0578, 0.0599, 0.0450, 0.0459,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0483, 0.0339, 0.0574, 0.0471, 0.0475, 0.0760, 0.0669, 0.0454, 0.0702,
         0.0543, 0.0936, 0.0745, 0.0420, 0.0396, 0.0450, 0.0637, 0.0406, 0.0538,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0507, 0.0534, 0.0739, 0.0597, 0.0445, 0.0463, 0.0600, 0.0526, 0.0671,
         0.0676, 0.0529, 0.0496, 0.0527, 0.0444, 0.0579, 0.0447, 0.0539, 0.0681,
       

In [28]:
len(A[0][0])

30

In [29]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(gelu(self.fc1(x)))


In [30]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

### 7. BERT

In [31]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_model, d_model]
        # it will be decided by first token(CLS)
        h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]

        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        # get masked position from final output of transformer.
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_clsf

### 8. Train Our Model

In [32]:
model = BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

for epoch in range(500):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 0010 cost = 152.215408
Epoch: 0020 cost = 26.173033
Epoch: 0030 cost = 16.047791
Epoch: 0040 cost = 9.183825
Epoch: 0050 cost = 5.328363
Epoch: 0060 cost = 4.378877
Epoch: 0070 cost = 3.558329
Epoch: 0080 cost = 2.709712
Epoch: 0090 cost = 2.611641
Epoch: 0100 cost = 2.553239
Epoch: 0110 cost = 2.575257
Epoch: 0120 cost = 2.496284
Epoch: 0130 cost = 2.577136
Epoch: 0140 cost = 2.516049
Epoch: 0150 cost = 2.566643
Epoch: 0160 cost = 2.500648
Epoch: 0170 cost = 2.576819
Epoch: 0180 cost = 2.489408
Epoch: 0190 cost = 2.458517
Epoch: 0200 cost = 2.493874
Epoch: 0210 cost = 2.515061
Epoch: 0220 cost = 2.434853
Epoch: 0230 cost = 2.478426
Epoch: 0240 cost = 2.450401
Epoch: 0250 cost = 2.410520
Epoch: 0260 cost = 2.408782
Epoch: 0270 cost = 2.464524
Epoch: 0280 cost = 2.284095
Epoch: 0290 cost = 2.049003
Epoch: 0300 cost = 2.340172
Epoch: 0310 cost = 2.090949
Epoch: 0320 cost = 1.816317
Epoch: 0330 cost = 1.869203
Epoch: 0340 cost = 1.804448
Epoch: 0350 cost = 2.099547
Epoch: 0360 cost

### 9. Inference

In [33]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)

Hello, how are you? I am Ali.
Hello, Ali My name is wael. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, wael
Thank you Ali
['[CLS]', 'great', 'my', 'baseball', 'team', '[MASK]', 'the', 'competition', '[SEP]', 'hello', 'how', 'are', '[MASK]', 'i', '[MASK]', 'ali', '[SEP]']
masked tokens list :  [12, 5, 23]
predict masked tokens list :  [25, 25, 25]
isNext :  False
predict isNext :  False


In [33]:
not good in MLM >>> need more epochs
but good in NSP